<a href="https://colab.research.google.com/github/jscienciadados/visualizacao-dados-plotly/blob/main/06_Sunburst_Candlesticks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# <font color='blue'>Visualização de Dados e Design de Dashboards</font>
## <font color='blue'>Capítulo 2 - Métodos de Visualização</font>

## Sunburst Diagram e Candlesticks com Plotly

## Pacotes Python Para Manipulação de Dados e Visualização

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
# Manipulação de Dados
import numpy as np
import pandas as pd
import datetime
from datetime import date, timedelta

# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science " --iversions

Author: Data Science 

IPython: 5.5.0
numpy  : 1.19.5
plotly : 4.4.1
pandas : 1.1.5



## Carregando os Dados

In [4]:
# Carrega os dados
titanic = pd.read_csv('dados/titanic-train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Carrega os dados
bajaj = pd.read_csv('dados/nifty50-stock-market-data-BAJAJFINSV.csv')
bajaj.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2008-05-26,BAJAJFINSV,EQ,2101.05,600.00,619.00,501.0,505.1,509.10,548.85,3145446,1.726368e+14,NaN,908264,0.2888
1,2008-05-27,BAJAJFINSV,EQ,509.10,505.00,610.95,491.1,564.0,554.65,572.15,4349144,2.488370e+14,NaN,677627,0.1558
2,2008-05-28,BAJAJFINSV,EQ,554.65,564.00,665.60,564.0,643.0,640.95,618.37,4588759,2.837530e+14,NaN,774895,0.1689
3,2008-05-29,BAJAJFINSV,EQ,640.95,656.65,703.00,608.0,634.5,632.40,659.60,4522302,2.982921e+14,NaN,1006161,0.2225
4,2008-05-30,BAJAJFINSV,EQ,632.40,642.40,668.00,588.3,647.0,644.00,636.41,3057669,1.945929e+14,NaN,462832,0.1514


### Sunburst

**Objetivo**: Exibir a representação quantitativa de muitas variáveis categóricas por distribuição de tamanho.

Descreva a distribuição entre as categorias de idade, sexo e pessoas sobreviventes no que diz respeito à tarifa (fare) no Titanic.

In [6]:
# Prepara os dados
titanic = titanic.dropna()
titanic['age_category'] = np.where((titanic['Age'] < 19), "Abaixo de 19 Anos",
                                 np.where((titanic['Age'] > 18) & (titanic['Age'] <= 30), "19-30",
                                    np.where((titanic['Age'] > 30) & (titanic['Age'] <= 50), "31-50",
                                                np.where(titanic['Age'] > 50, "Acima de 50 Anos", "NULL"))))

age = titanic['age_category'].value_counts().to_frame().reset_index().rename(columns = {'index':'age_category','age_category':'Count'})

titanic['survived_or_not'] = np.where(titanic['Survived']==1,"Survived", np.where(titanic['Survived']==0,"Died","null")) 

sun_df = titanic[['Sex','survived_or_not','Embarked','age_category','Cabin','Fare']].groupby(['Sex','survived_or_not','age_category','Embarked','Cabin']).agg('sum').reset_index()

In [10]:
# Figura
fig = px.sunburst(sun_df, path = ['Sex','survived_or_not','age_category'], values = 'Fare')

# Layout
fig.update_layout(title = "Distribuição de Passageiros Por Faixa Etária, Sexo e Sobrevivência, Por Tarifa", 
                  title_x = 0.5)

# Gráfico
fig.show()

### Sunburst Color

Mesmo gráfico anterior com diferente organização de cores por sobrevivente/não sobrevivente.

In [ ]:
# Figura
fig = px.sunburst(sun_df, path = ['Sex','survived_or_not','age_category'], 
                  values = 'Fare', 
                  color = 'survived_or_not',
                  color_discrete_map = {'(?)':'black', 'Died':'red', 'Survived':'darkblue'})

# Layout
fig.update_layout(title = "Distribuição de Passageiros Por Faixa Etária, Sexo e Sobrevivência, Por Tarifa",
                  title_x = 0.5)

# Gráfico
fig.show()

### Candlestick

**Objetivo**: Mostrar séries com pontos de alta, baixa, abertura e fechamento (usado para visualizar preços de ações).

Como o preço das ações da empresa Bajaj Finserv flutuou ao longo do tempo? Exiba com pontos de alta, baixa, abertura e fechamento.

In [8]:
# Figura
fig = go.Figure(data = [go.Candlestick(x = bajaj['Date'],
                                       open = bajaj['Open'],
                                       high = bajaj['High'],
                                       low = bajaj['Low'],
                                       close = bajaj['Close'])])

# Layout
fig.update_layout(title = "Candlestick Series do Preço de Ações da Empresa Bajaj Finsv")

# Gráfico
fig.show()

## Candlestick com Ponto Específico

O mesmo gráfico anterior com uma linha vertical indicando uma data específica.

In [9]:
# Figura
fig = go.Figure(data = [go.Candlestick(x = bajaj['Date'],
                                       open = bajaj['Open'],
                                       high = bajaj['High'],
                                       low = bajaj['Low'],
                                       close = bajaj['Close'])])

# Layout com linha específica
fig.update_layout(title = "Candlestick Series do Preço de Ações da Empresa Bajaj Finsv",
                  xaxis_rangeslider_visible = False,  
                  shapes = [dict(x0 = '2017-01-07', 
                                 x1 = '2017-01-07', 
                                 y0 = 0, 
                                 y1 = 1, 
                                 xref = 'x', 
                                 yref = 'paper',
                                 line_width = 2)],
                  annotations = [dict(x = '2017-01-07', 
                                      y = 0.05, 
                                      xref = 'x', 
                                      yref = 'paper',
                                      showarrow = False, 
                                      xanchor = 'left', 
                                      text = 'Fusão com Empresas Aquiridas')])

# Gráfico
fig.show()

# Fim